In [2]:
import sqlite3
import tkinter as tk
from tkinter import messagebox
import logging
import sys

# Step 1: Create/connect to SQLite database
conn = sqlite3.connect("soulsense_db")
cursor = conn.cursor()

# Step 2: Create table to store final score
cursor.execute("""
CREATE TABLE IF NOT EXISTS scores (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT,
    age INTEGER,
    total_score INTEGER
)

# Step 3: Import questions from question_bank file
questions = []
with open('question_bank.txt', 'r') as question_bank:
    questions = [line.strip() for line in question_bank]

# ---------------- GUI APPLICATION ----------------
class SoulSenseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Soul Sense EQ Test")
        self.username = ""
        self.age = None  # Optional age field
        self.current_question = 0
        self.responses = []

        self.create_username_screen()

    def create_username_screen(self):
        self.clear_screen()

        tk.Label(self.root, text="Enter Your Name:", font=("Arial", 14)).pack(pady=20)
        self.name_entry = tk.Entry(self.root, font=("Arial", 14))
        self.name_entry.pack(pady=10)

        tk.Label(self.root, text="Enter Your Age (optional):", font=("Arial", 14)).pack(pady=10)
        self.age_entry = tk.Entry(self.root, font=("Arial", 14))
        self.age_entry.pack(pady=10)

        tk.Button(self.root, text="Start Test", command=self.start_test, font=("Arial", 12)).pack(pady=20)

    def validate_name_input(self, name_str):
        """Validate name input - returns (is_valid, error_message)"""
        if name_str == "":
            return (False, "Please enter your name to start the test.")
        if len(name_str) < 2:
            return (False, "Name must be at least 2 characters long.")
        if len(name_str) > 50:
            return (False, "Name must be less than 50 characters.")
        if not all(char.isalpha() or char.isspace() for char in name_str):
            return (False, "Name should contain only letters and spaces. No numbers or special characters.")
        return (True, None)

    def validate_age_input(self, age_str):
        """Validate age input - returns (is_valid, age_value, error_message)"""
        if age_str == "":
            return (True, None, None)  # Age is optional
        try:
            age = int(age_str)
            if age < 1 or age > 120:
                return (False, None, "Please enter a valid age between 1 and 120.")
            return (True, age, None)
        except ValueError:
            return (False, None, "Age must be a numeric value. Please enter a valid number.")

    def start_test(self):
        self.username = self.name_entry.get().strip()
        age_str = self.age_entry.get().strip()
        
        # Validate name input
        is_valid, error_msg = self.validate_name_input(self.username)
        if not is_valid:
            messagebox.showwarning("Input Error", error_msg)
            return
        
        # Validate age input
        is_valid, age_value, error_msg = self.validate_age_input(age_str)
        if not is_valid:
            messagebox.showwarning("Input Error", error_msg)
            return
        
        self.age = age_value
        self.show_question()

    def show_question(self):
        self.clear_screen()

        try:
            if self.current_question < len(questions):
                q_text = questions[self.current_question]

                tk.Label(
                    self.root,
                    text=f"Q{self.current_question + 1}: {q_text}",
                    wraplength=400,
                    font=("Arial", 14)
                ).pack(pady=20)

                self.answer_var = tk.IntVar()

                for val, text in enumerate(
                    ["Never (1)", "Sometimes (2)", "Often (3)", "Always (4)"], start=1
                ):
                    tk.Radiobutton(
                        self.root,
                        text=text,
                        variable=self.answer_var,
                        value=val,
                        font=("Arial", 12)
                    ).pack(anchor="w", padx=50)

                tk.Button(
                    self.root,
                    text="Next",
                    command=self.save_answer,
                    font=("Arial", 12)
                ).pack(pady=20)

            else:
                self.finish_test()

        except Exception:
            logging.error("Question loading failed", exc_info=True)
            messagebox.showerror("Error", "An error occurred while loading questions.")

    def save_answer(self):
        ans = self.answer_var.get()

        if ans == 0:
            logging.warning(
                f"Input validation failed: No answer selected (Q{self.current_question + 1})"
            )
            messagebox.showwarning("Input Error", "Please select an answer before proceeding.")
            return

        self.responses.append(
            (self.username, self.current_question + 1, questions[self.current_question], ans)
        )

        self.current_question += 1
        self.show_question()

    def finish_test(self):
        total_score = sum(r[3] for r in self.responses)

        # Store only final score in the database
        cursor.execute("INSERT INTO scores (username, age, total_score) VALUES (?, ?, ?)", (self.username, self.age, total_score))
        conn.commit()

            logging.info(
                f"Score saved successfully | user={self.username} | score={total_score}"
            )

        except Exception:
            logging.error("Database insert failed while saving final score", exc_info=True)
            messagebox.showerror(
                "Database Error",
                "Unable to save your score. Please try again."
            )
            return

        interpretation = (
            "Excellent Emotional Intelligence!" if total_score >= 65 else
            "Good Emotional Intelligence." if total_score >= 50 else
            "Average Emotional Intelligence." if total_score >= 35 else
            "You may want to work on your Emotional Intelligence."
        )

        self.clear_screen()

        tk.Label(self.root, text=f"Thank you, {self.username}!", font=("Arial", 16)).pack(pady=10)
        tk.Label(
            self.root,
            text=f"Your total EQ score is: {total_score} / 80",
            font=("Arial", 14)
        ).pack(pady=10)
        tk.Label(
            self.root,
            text=interpretation,
            font=("Arial", 14),
            fg="blue"
        ).pack(pady=10)

        tk.Button(self.root, text="Exit", command=self.exit_test, font=("Arial", 12)).pack(pady=20)

    def show_all_results(self):
        # Display all results in the command line
        print("\n--- All EQ Test Results ---")
        print(f"{'Username':<20} {'Age':<10} {'Total Score'}")
        print("-" * 45)

        # Fetch all scores from the database
        cursor.execute("SELECT username, age, total_score FROM scores")
        rows = cursor.fetchall()

        for row in rows:
            age_display = str(row[1]) if row[1] is not None else "N/A"
            print(f"{row[0]:<20} {age_display:<10} {row[2]}")

    def exit_test(self):
        try:
            conn.close()
            logging.info("Database connection closed")
        except Exception:
            logging.error("Error closing database connection", exc_info=True)

        logging.info("Application exited gracefully")
        self.root.quit()

    def clear_screen(self):
        for widget in self.root.winfo_children():
            widget.destroy()

# ---------------- MAIN LOOP ----------------
if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("500x300")

    app = SoulSenseApp(root)
    root.protocol("WM_DELETE_WINDOW", app.exit_test)

    root.mainloop()
